In [17]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv("data.csv")

In [9]:
df.columns

Index(['year', 'commodity', 'port', 'value', 'type', 'unit', 'quantity'], dtype='object')

In [11]:
df.port.unique()

array(['Bengal', 'Madras', 'Bombay', 'St_Helena', 'Benkulen', 'China',
       'Angola', 'Penang', 'Java', 'Madagascar', 'Mocha', 'Madeira',
       'Cape', 'Singapore', 'Canton', 'Persia', 'Balambangan'],
      dtype=object)

In [13]:
port_coordinates = {
    'Bengal': (22.5726, 88.3639),  # Kolkata (historical area of Bengal)
    'Madras': (13.0827, 80.2707),  # Chennai (historically known as Madras)
    'Bombay': (19.0760, 72.8777),  # Mumbai (historically known as Bombay)
    'St_Helena': (-15.9650, -5.7089),
    'Benkulen': (-3.8004, 102.2655),  # Bengkulu
    'China': (35.8617, 104.1954),  # Approximate central China, need specific port
    'Angola': (-11.2027, 17.8739),  # Approximate center of Angola, need specific port
    'Penang': (5.4141, 100.3288),
    'Java': (-7.6145, 110.7122),  # Approximate center of Java
    'Madagascar': (-18.7669, 46.8691),  # Approximate center of Madagascar
    'Mocha': (13.3250, 43.2481),
    'Madeira': (32.7607, -16.9595),
    'Cape': (-33.9249, 18.4241),  # Cape Town
    'Singapore': (1.3521, 103.8198),
    'Canton': (23.1291, 113.2644),  # Guangzhou (historically known as Canton)
    'Persia': (32.4279, 53.6880),  # Approximate center of Persia (Iran)
    'Balambangan': (6.1656, 116.7244),
}

In [14]:
df['coords'] = df['port'].map(port_coordinates)

In [15]:
df

,year,commodity,port,value,type,unit,quantity,coords
0,1760,Broadcloth,Bengal,26263.0,export,cloths,2595,"(22.5726, 88.3639)"
1,1760,Broadcloth,Madras,7564.0,export,cloths,600,"(13.0827, 80.2707)"
2,1760,Broadcloth,Bombay,64038.0,export,cloths,4712,"(19.076, 72.8777)"
3,1760,Broadcloth,St_Helena,177.0,export,cloths,6,"(-15.965, -5.7089)"
4,1760,Broadcloth,Benkulen,576.0,export,cloths,28,"(-3.8004, 102.2655)"
...,...,...,...,...,...,...,...,...
6109,1833,Textiles,China,0.0,import,pieces,0,"(35.8617, 104.1954)"
6110,1834,Textiles,Bengal,59331.0,import,pieces,60217,"(22.5726, 88.3639)"
6111,1834,Textiles,Madras,0.0,import,pieces,0,"(13.0827, 80.2707)"
6112,1834,Textiles,Bombay,0.0,import,pieces,0,"(19.076, 72.8777)"


In [18]:
selected_year = 1760
data_1760 = df[df['year'] == selected_year]
trade_summary = data_1760.groupby(['port', 'type']).agg({'value': 'sum', 'quantity': 'sum'}).reset_index()

In [19]:
base_map = folium.Map(location=[0, 80], zoom_start=3)

# Add ports as markers on the map
for port, coords in port_coordinates.items():
    if port in trade_summary['port'].values:
        folium.Marker(
            location=coords,
            popup=f"{port}\nExport Value: {trade_summary[trade_summary['port'] == port]['value'].sum():,.0f}\nExport Quantity: {trade_summary[trade_summary['port'] == port]['quantity'].sum():,.0f}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(base_map)

In [21]:
base_map.save('1760_trade_map.html')


In [30]:
df[(df['year'] == 1800) & (df['commodity'] == 'Silver')]

,year,commodity,port,value,type,unit,quantity,coords
4566,1800,Silver,Bengal,0.0,export,ozs,0,"(22.5726, 88.3639)"
4567,1800,Silver,Madras,0.0,export,ozs,0,"(13.0827, 80.2707)"
4568,1800,Silver,Bombay,100196.0,export,ozs,380192,"(19.076, 72.8777)"
4569,1800,Silver,St_Helena,0.0,export,ozs,0,"(-15.965, -5.7089)"
4570,1800,Silver,Mocha,0.0,export,ozs,0,"(13.325, 43.2481)"
4571,1800,Silver,Benkulen,0.0,export,ozs,0,"(-3.8004, 102.2655)"
4572,1800,Silver,Canton,140733.0,export,ozs,530298,"(23.1291, 113.2644)"


In [10]:
import dash
from dash import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [6]:
data = pd.read_csv("data.csv")

In [15]:
app = dash.Dash(__name__)

commodities = [{'label': commodity, 'value': commodity} for commodity in data['commodity'].unique()]
ports = [{'label': port, 'value': port} for port in data['port'].unique()]
trade_types = [{'label': trade_type, 'value': trade_type} for trade_type in data['type'].unique()]



app.layout = html.Div([
    html.H1("Trade Data Dashboard", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Select Commodity:"),
        dcc.Dropdown(id='commodity-dropdown', options=commodities, value=commodities[0]['value']),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Select Port:"),
        dcc.Dropdown(id='port-dropdown', options=ports, value=ports[0]['value']),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Select Trade Type:"),
        dcc.Dropdown(id='type-dropdown', options=trade_types, value=trade_types[0]['value']),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    dcc.Graph(id='trade-volume-graph'),
    
    html.Div([
        html.Label("Select Port:"),
        dcc.Dropdown(
            id='port-selection-dropdown',
            options=[{'label': port, 'value': port} for port in data['port'].unique()],
            value=data['port'].unique()[0]  # Default to the first port
        ),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    dash_table.DataTable(
        id='trade-summary-table',
        columns=[
            {'name': 'Year', 'id': 'year'},
            {'name': 'Commodity', 'id': 'commodity'},
            {'name': 'Type', 'id': 'type'},
            {'name': 'Total Value', 'id': 'value'},
            {'name': 'Total Quantity', 'id': 'quantity'},
        ],
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            # all three widths are needed
            'minWidth': '80px', 'width': '80px', 'maxWidth': '80px',
            'whiteSpace': 'normal'
        },
    )
])






In [16]:
# Trade Volume

@app.callback(
    Output('trade-volume-graph', 'figure'),
    [Input('commodity-dropdown', 'value'),
     Input('port-dropdown', 'value'),
     Input('type-dropdown', 'value')]
)

def update_graph(selected_commodity, selected_port, selected_type):
    filtered_data = data[
        (data['commodity'] == selected_commodity) &
        (data['port'] == selected_port) &
        (data['type'] == selected_type)
    ]
    
    if filtered_data.empty:
        fig = px.line(title="No Data Available")
    else:
        fig = px.line(filtered_data, x='year', y='quantity',
                      title=f'Trade Volume for {selected_commodity} in {selected_port} ({selected_type})')
        fig.update_xaxes(title_text='Year')
        fig.update_yaxes(title_text='Quantity')
    
    return fig

In [19]:
# Trade Summary

@app.callback(
    Output('trade-summary-table', 'data'),
    [Input('port-selection-dropdown', 'value')]
)

def update_trade_summary(selected_port):
    filtered_data = data[data['port'] == selected_port]
    
    # Aggregate trade data for the selected port
    aggregated_data = filtered_data.groupby(['commodity', 'type']).agg({'value': 'sum', 'quantity': 'sum'}).reset_index()
    
    # Sort the data by year for better readability
    aggregated_data.sort_values('year', inplace=True)
    

    return aggregated_data.to_dict('records')


In [20]:
if __name__ == '__main__':
    app.run_server(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [50]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load data and preprocess
data = pd.read_csv("data.csv")
data = data[data['quantity'] != 0]

# Use a dark theme from Dash Bootstrap Components
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

# Prepare dropdown options for commodities
commodities = [{'label': commodity, 'value': commodity} for commodity in data['commodity'].unique()]

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Trade From London (1760 - 1830)", className="text-center text-primary, mb-4"))),
    
    dbc.Row(dbc.Col([
        html.Label("Select Commodity:", className="text-light"),
        dcc.Dropdown(id='commodity-dropdown', options=commodities, value=commodities[0]['value']),
    ], width=12)),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='trade-trends-graph'), width=6),
        dbc.Col(dcc.Graph(id='commodity-value-graph'), width=6)
    ]),
    
    html.H2("Sum Of All Trade", className="text-center text-light, mb-4"),
    
    dbc.Row(dbc.Col(dash_table.DataTable(
        id='trade-breakdown-table',
        columns=[
            {'name': 'Port', 'id': 'port'},
            {'name': 'Type', 'id': 'type'},
            {'name': 'Total Value', 'id': 'value'},
            {'name': 'Quantity', 'id': 'quantity'}
        ],
        style_table={'overflowX': 'auto'},
        style_cell={
            'color': 'white',
            'minWidth': '80px', 'width': '120px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'border': '1px solid grey'
        },
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'border': '1px solid grey'
        },
    ), width=12)),
], fluid=True)

@app.callback(
    Output('trade-trends-graph', 'figure'),
    [Input('commodity-dropdown', 'value')]
)
def update_trade_trends(selected_commodity):
    filtered_data = data[data['commodity'] == selected_commodity]

    # Creating a new column 'port_type' to concatenate 'port' and 'type' for unique line labels
    filtered_data['port_type'] = filtered_data['port'] + " (" + filtered_data['type'] + ")"

    fig = px.line(filtered_data, x='year', y='quantity', color='port_type',
                  title=f'Quantity for {selected_commodity} by Year, Port, and Type')
    fig.update_xaxes(title_text='Year')
    fig.update_yaxes(title_text='Quantity')
    return fig

@app.callback(
    Output('commodity-value-graph', 'figure'),
    [Input('commodity-dropdown', 'value')]
)
def update_commodity_price(selected_commodity):
    filtered_data = data[data['commodity'] == selected_commodity]

    unit_name = filtered_data['unit'].values[0]
    # Creating a new column 'port_type' to concatenate 'port' and 'type' for unique line labels
    filtered_data['price_per_unit'] = filtered_data['quantity'] / filtered_data['value']

    fig = px.line(filtered_data, x='year', y='price_per_unit', color='port',
                  title=f'Price per {unit_name} for {selected_commodity} by Year, and Port')
    fig.update_xaxes(title_text='Year')
    fig.update_yaxes(title_text=f'Price Per {unit_name}')
    return fig

@app.callback(
    Output('trade-breakdown-table', 'data'),
    [Input('commodity-dropdown', 'value')]
)
def update_trade_breakdown(selected_commodity):
    filtered_data = data[data['commodity'] == selected_commodity]
    
    # No need to adjust for 'destination_source' as previously, use 'port' directly
    aggregated_data = filtered_data.groupby(['port', 'type']).agg({
        'value': 'sum',
        'quantity': 'sum'
    }).reset_index()
    
    return aggregated_data.to_dict('records')

if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-50-c551d3b978d4>:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-c551d3b978d4>:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-c551d3b978d4>:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-c551

In [37]:
filtered_data = data[data['commodity'] == 'Tea']

In [41]:
filtered_data['price_per_unit'] = filtered_data['quantity'] / filtered_data['value']

<ipython-input-41-5c80b2bbd117>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
filtered_data['unit'].values[0]

'lbs'